This code is loading both the json file contaning all code snippents and there real label. It also loads the csv file that contains all of my collected CVE vulnerability summaries.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00
ERROR: pip's de

In [4]:
import os
import json
import concurrent.futures
import pandas as pd
import random
import torch
import transformers
import torch
import re
from google.colab import userdata
from datasets import load_dataset
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, logging
from transformers import BitsAndBytesConfig, GemmaTokenizer, pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

This code block loads in our processed DiverseVul dataset in a json format.

In [5]:
# Define the paths to your JSON files
code_data_path = "/content/drive/MyDrive/processedData.json"

# Load the data
corrected_data = []
with open(code_data_path, 'r') as file:
    for line in file:
        try:
            json_object = json.loads(line)
            corrected_data.append(json_object)
        except json.JSONDecodeError:
            print("Error decoding JSON from line:", line)

# Pretty print the first JSON object
print(json.dumps(corrected_data[:5], indent=4))

[
    {
        "func": "int _gnutls_ciphertext2compressed(gnutls_session_t session,\n\t\t\t\t  opaque * compress_data,\n\t\t\t\t  int compress_size,\n\t\t\t\t  gnutls_datum_t ciphertext, uint8 type)\n{\n    uint8 MAC[MAX_HASH_SIZE];\n    uint16 c_length;\n    uint8 pad;\n    int length;\n    mac_hd_t td;\n    uint16 blocksize;\n    int ret, i, pad_failed = 0;\n    uint8 major, minor;\n    gnutls_protocol_t ver;\n    int hash_size =\n\t_gnutls_hash_get_algo_len(session->security_parameters.\n\t\t\t\t  read_mac_algorithm);\n\n    ver = gnutls_protocol_get_version(session);\n    minor = _gnutls_version_get_minor(ver);\n    major = _gnutls_version_get_major(ver);\n\n    blocksize = _gnutls_cipher_get_block_size(session->security_parameters.\n\t\t\t\t\t      read_bulk_cipher_algorithm);\n\n    /* initialize MAC \n     */\n    td = mac_init(session->security_parameters.read_mac_algorithm,\n\t\t  session->connection_state.read_mac_secret.data,\n\t\t  session->connection_state.read_mac_secret

This code block loads in our privously saved CWE summarys as JSON.

In [6]:
# Define the path to your JSON file
cwe_data = "/content/drive/MyDrive/cwe_data.json"

# Load the sorted unique CWE descriptions
cwe_descriptions = []
with open(cwe_data, 'r') as file:
    for line in file:
        cwe_descriptions.append(json.loads(line))

# Print sample description
print("\nCWE Description Sample:")
print(json.dumps(cwe_descriptions[:5], indent=4))


CWE Description Sample:
[
    {
        "CWE ID": "CWE-1",
        "CWE Name": "Location",
        "Description": "Description SummaryWeaknesses in this category are organized based on which phase they are introduced during the software development and deployment process."
    },
    {
        "CWE ID": "CWE-2",
        "CWE Name": "Environment",
        "Description": "Description SummaryWeaknesses in this category are typically introduced during unexpected environmental conditions."
    },
    {
        "CWE ID": "CWE-3",
        "CWE Name": "Technology-specific Environment Issues",
        "Description": "Description SummaryWeaknesses in this category are typically introduced during unexpected environmental conditions in particular technologies."
    },
    {
        "CWE ID": "CWE-4",
        "CWE Name": "J2EE Environment Issues",
        "Description": "Description SummaryJ2EE framework related environment issues with security implications."
    },
    {
        "CWE ID": "CWE-5"

Example of what a DiverseVul entry with a CWE looks like for filtering purposes.

In [17]:
labeled_data = [entry for entry in corrected_data if entry.get('target') == 1 and entry.get('cwe')]
print(f'Number of CWE Labeled Functions: {len(labeled_data)}. Compared to the total functions of: {len(corrected_data)}')
print(json.dumps(labeled_data[:5], indent=4))

Number of CWE Labeled Functions: 16109. Compared to the total functions of: 330492
[
    {
        "func": "static char *make_filename_safe(const char *filename TSRMLS_DC)\n{\n\tif (*filename && strncmp(filename, \":memory:\", sizeof(\":memory:\")-1)) {\n\t\tchar *fullpath = expand_filepath(filename, NULL TSRMLS_CC);\n\n\t\tif (!fullpath) {\n\t\t\treturn NULL;\n\t\t}\n\n\t\tif (PG(safe_mode) && (!php_checkuid(fullpath, NULL, CHECKUID_CHECK_FILE_AND_DIR))) {\n\t\t\tefree(fullpath);\n\t\t\treturn NULL;\n\t\t}\n\n\t\tif (php_check_open_basedir(fullpath TSRMLS_CC)) {\n\t\t\tefree(fullpath);\n\t\t\treturn NULL;\n\t\t}\n\t\treturn fullpath;\n\t}\n\treturn estrdup(filename);\n}",
        "target": 1,
        "cwe": [
            "CWE-264"
        ],
        "message": "Improve check for :memory: pseudo-filename in SQlite"
    },
    {
        "func": "static void cirrus_do_copy(CirrusVGAState *s, int dst, int src, int w, int h)\n{\n    int sx, sy;\n    int dx, dy;\n    int width, height;\n   

This code block is taking both the DiverseVul functions, and the CWE summarys and combining them into a dictionary of matching pairs. This way we can later do one to one testing to see how accuratly a LLM can say if the function and code correspond.

Now take the labeled data and make a new JSON file that has every vulnerable function with a correlating summary.

In [19]:
output_path = "/content/drive/MyDrive/summary_pairs.json"

# Load the CWE descriptions into a dictionary.
cwe_summaries = {}
for item in cwe_descriptions:
  cwe_summaries[item['CWE ID']] = item['Description']

# Load the matching functions.
output_data = []
unmatched_functions = 0
for entry in labeled_data:
    summaries = []
    for cwe_id in entry['cwe']:  # Loop through each CWE ID in the list
        if cwe_id in cwe_summaries:
            summary = cwe_summaries[cwe_id]
            clean_summary = summary.replace("Description Summary", "").strip()
            summaries.append(clean_summary)
    if summaries:
        combined_summary = " ".join(summaries)
        id = entry['cwe']
        output_data.append({'summary': combined_summary, 'CWE': id, 'func': entry['func']})
    else:
        unmatched_functions += 1

# Save to a new JSON file
with open(output_path, 'w') as json_file:
    json.dump(output_data, json_file, ensure_ascii=True, indent=4)

# Print results for verification
print(json.dumps(output_data[:3], indent=4))
print(f"Combined Length: {len(output_data)}")
print(f"Summarys Length: {len(cwe_descriptions)}")
print(f"Dictionary Length: {len(cwe_summaries)}")
print(f"Functions Length: {len(labeled_data)}")
print(f"Number of unmatched functions: {unmatched_functions}")

[
    {
        "summary": "Weaknesses in this category are related to the management of permissions, privileges, and other security features that are used to perform access control.",
        "CWE": [
            "CWE-264"
        ],
        "func": "static char *make_filename_safe(const char *filename TSRMLS_DC)\n{\n\tif (*filename && strncmp(filename, \":memory:\", sizeof(\":memory:\")-1)) {\n\t\tchar *fullpath = expand_filepath(filename, NULL TSRMLS_CC);\n\n\t\tif (!fullpath) {\n\t\t\treturn NULL;\n\t\t}\n\n\t\tif (PG(safe_mode) && (!php_checkuid(fullpath, NULL, CHECKUID_CHECK_FILE_AND_DIR))) {\n\t\t\tefree(fullpath);\n\t\t\treturn NULL;\n\t\t}\n\n\t\tif (php_check_open_basedir(fullpath TSRMLS_CC)) {\n\t\t\tefree(fullpath);\n\t\t\treturn NULL;\n\t\t}\n\t\treturn fullpath;\n\t}\n\treturn estrdup(filename);\n}"
    },
    {
        "summary": "The software writes data past the end, or before the beginning, of the intended buffer. Extended Description  This typically occurs when the p

In [9]:
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [20]:
# Set logging to error to avoid repetitive log messages
logging.set_verbosity_error()

# Load the model and tokenizer
model_id_llama = "meta-llama/Meta-Llama-3-8B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [21]:
tokenizer_llama = AutoTokenizer.from_pretrained(model_id_llama, token=os.environ['HF_TOKEN'], force_download=True)

# Explicitly set the padding token to avoid repetitive internal checks and logs
if tokenizer_llama.pad_token is None:
    tokenizer_llama.pad_token = tokenizer_llama.eos_token

model_llama = AutoModelForCausalLM.from_pretrained(model_id_llama,
                                                   quantization_config=bnb_config,
                                                   device_map={"":0},
                                                   token=os.environ['HF_TOKEN'],
                                                   force_download=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
def analyze_summary_function_pair(summary, function):
    """Generate the prompt and analyze if the summary matches the function."""
    prompt = f"Summary: '{summary}'. Does the following function have the described vulnerability? \nFunction: {function}"
    inputs = tokenizer_llama(prompt, return_tensors="pt", truncation=True).to('cuda')
    outputs = model_llama.generate(**inputs, max_new_tokens=356, do_sample=True, temperature=0.2, top_k=20, top_p=0.95)
    result = tokenizer_llama.decode(outputs[0], skip_special_tokens=True)
    return result.strip()

# Sample summary and function for the test
summary = "Buffer overflow vulnerability occurs when data exceeds buffer boundaries, allowing attackers to execute arbitrary code or cause a segmentation fault in the system."
function = """
void vulnerable_function(char *input) {
    char buffer[10];
    strcpy(buffer, input);  // This is vulnerable to buffer overflow
}
"""

# Generate a prediction
prediction = analyze_summary_function_pair(summary, function)
print("Prediction:", prediction)

Prediction: Summary: 'Buffer overflow vulnerability occurs when data exceeds buffer boundaries, allowing attackers to execute arbitrary code or cause a segmentation fault in the system.'. Does the following function have the described vulnerability? 
Function: 
void vulnerable_function(char *input) {
    char buffer[10];
    strcpy(buffer, input);  // This is vulnerable to buffer overflow
}
Explanation: 
The function `vulnerable_function` takes a character pointer `input` as an argument. It then declares a character array `buffer` of size 10. The `strcpy` function is used to copy the contents of `input` to `buffer`. This is where the buffer overflow vulnerability occurs. If the length of `input` is greater than 10, the extra characters will overflow the `buffer` array, potentially allowing an attacker to execute arbitrary code or cause a segmentation fault in the system. 

To fix this vulnerability, you can use functions like `strncpy` or `memcpy` to limit the amount of data copied to 

In [29]:
def analyze_summary_function_pair(summary, function):
    prompt = f"""
    Given the following summary and function, determine if the function contains the described vulnerability.
    Summary: '{summary}'

    Function:
    {function}

    Is the function vulnerable? Yes or No. Explain your answer in one sentence.
    """
    inputs = tokenizer_llama(prompt, return_tensors="pt", truncation=True).to('cuda')
    outputs = model_llama.generate(**inputs, max_new_tokens=50, do_sample=True, temperature=0.05, top_k=15, top_p=0.95)
    result = tokenizer_llama.decode(outputs[0], skip_special_tokens=True)
    # Extract the model's response by finding the part after the specific question
    split_result = result.split("Is the function vulnerable? Yes or No. Explain your answer in one sentence.")
    if len(split_result) > 1:
        response = split_result[-1].strip()
    else:
        response = "Uncertain"

    return response

for item in output_data[:3]:
    summary = item['summary']
    function = item['func']

    torch.cuda.empty_cache()  # Clear cache before each operation
    print(f"Processing summary: {summary[:50]}...")  # Preview of summary being processed
    prediction = analyze_summary_function_pair(summary, function)
    print(f"Summary: {summary}")
    print(f"Prediction: {prediction}\n")

Processing summary: Weaknesses in this category are related to the man...
Summary: Weaknesses in this category are related to the management of permissions, privileges, and other security features that are used to perform access control.
Prediction: Yes. The function does not properly validate the input filename, allowing for potential directory traversal attacks. 

    Note: The function checks if the filename starts with ":memory:", but it does not check if the filename contains any other malicious characters.

Processing summary: The software writes data past the end, or before t...
Summary: The software writes data past the end, or before the beginning, of the intended buffer. Extended Description  This typically occurs when the pointer or its index is incremented or decremented to a position beyond the bounds of the buffer or when pointer arithmetic results in a position outside of the valid memory location to name a few. This may result in corruption of sensitive information, a c